# FAC: Field-aligned currents

In [ ]:
import datetime as dt
import numpy as np
from swarmpal.io import create_paldata, PalDataItem
from swarmpal.toolboxes.fac import FAC_singlesat

## Fetching data

In [ ]:
data_params = dict(
    collection="SW_OPER_MAGA_LR_1B",
    measurements=["B_NEC"],
    models=["CHAOS"],
    start_time="2016-01-01T00:00:00",
    end_time="2016-01-01T03:00:00",
    server_url="https://vires.services/ows",
    options=dict(asynchronous=False, show_progress=False),
)

data = create_paldata(**{"sample/alpha": PalDataItem.from_vires(**data_params)})
print(data)

## Applying a process and viewing the results

In [ ]:
process = FAC_singlesat(
    active_tree="sample",
    config={
        "dataset": "alpha",
        "model_varname": "B_NEC_CHAOS",
        "measurement_varname": "B_NEC",
    },
)
data = data.swarmpal.apply(process)
print(data)

In [ ]:
data.swarmpal.pal_meta

In [ ]:
data.swarmpal_fac.quicklook();

## Retrying with data subselection

This time we will fetch data with a filter applied to the request from VirES

See [viresclient.SwarmRequest.add_filter](https://viresclient.readthedocs.io/en/latest/api.html#viresclient.SwarmRequest.add_filter) for how these behave. `swarmpal.io.PalDataItem.from_vires` accepts a list of such filters.

NB. there is currently a bug requiring that each filter string is enclosed in parentheses.

In [ ]:
data_params = dict(
    collection="SW_OPER_MAGA_LR_1B",
    measurements=["B_NEC"],
    models=["CHAOS"],
    start_time="2016-01-01T00:00:00",
    end_time="2016-01-01T03:00:00",
    server_url="https://vires.services/ows",
    options=dict(asynchronous=False, show_progress=False),
    filters=[
        "((QDLat > 50) OR (QDLat < -50))",  # the algorithm is only valid at high latitude
        "(Flags_B <= 9)",  # Exclude particularly bad data
    ],
)

data = create_paldata(**{"sample/alpha": PalDataItem.from_vires(**data_params)})
data = data.swarmpal.apply(process)
data.swarmpal_fac.quicklook();